In [79]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

from uszipcode import SearchEngine
from  geopy.distance import distance




import warnings; warnings.simplefilter('ignore')

In [80]:
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [81]:

!dir .\Data\*.tsv

 Volume in drive D is DATA
 Volume Serial Number is 541D-B1F5

 Directory of D:\Github\Jobs_Recommender\Data

08/01/2012  07:36 PM        75,029,533 apps.tsv
08/09/2012  01:32 PM     3,403,585,644 jobs.tsv
08/09/2012  01:34 PM       893,351,890 jobs1.tsv
08/09/2012  01:36 PM       484,177,559 jobs2.tsv
08/09/2012  01:37 PM       434,304,750 jobs3.tsv
08/09/2012  01:39 PM       434,975,972 jobs4.tsv
08/09/2012  01:40 PM       438,540,834 jobs5.tsv
08/09/2012  01:41 PM       358,506,865 jobs6.tsv
08/09/2012  01:43 PM       359,728,308 jobs7.tsv
06/29/2019  11:39 PM           234,429 test_users.tsv
06/29/2019  11:39 PM        35,261,359 users.tsv
06/29/2019  11:39 PM        72,423,459 user_history.tsv
06/29/2019  11:39 PM               493 window_dates.tsv
              13 File(s)  6,990,121,095 bytes
               0 Dir(s)  204,962,562,048 bytes free


In [82]:
folder = 'Data'

apps = pd.read_csv('./'+folder+'/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./'+folder+'/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./'+folder+'/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./'+folder+'/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('./'+folder+'/test_users.tsv', delimiter='\t',encoding='utf-8')


b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


### EDA

In [83]:
apps.head()


,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [84]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [85]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [86]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [87]:
test_users.head()

,UserID,WindowID
0,767,1
1,769,1
2,861,1
3,1006,1
4,1192,1


#### Subsetting jobs in NY

In [88]:
jobs = jobs.loc[jobs.State == 'NY']
jobs.to_csv("NYjobs.tsv",  sep='\t',encoding='utf-8')

In [89]:
import gc
gc.collect()

98

In [90]:

jobs.groupby(['Zip5', 'City']).size().reset_index(name='Count').sort_values('Count', ascending=False).head()

,Zip5,City,Count
3,10001,New York,1908
1157,10001,New York,1382
29,10017,New York,588
27,10016,New York,453
1184,10017,New York,436


### Preprocessing

### Preprocessing Description and Requirements

In [106]:
import re

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [107]:

jobs['Description'] = jobs['Description'].astype(dtype='str').apply(preprocessor)

In [108]:
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(preprocessor)

### Creating jobs coordinates

In [97]:
search = SearchEngine(simple_zipcode=True)

In [96]:
jobs.Zip5 = jobs.Zip5.fillna(0)
jobs.Zip5 = jobs.Zip5.astype(int)

In [56]:
#lat, "long"
"""coords_1 = c
coords_2 = (34.1, -118.42)

print (round(distance(coords_1, coords_2).miles, 2))"""
search.by_zipcode("2e").lat

In [104]:

def coordinates(zipcode):
    zipcode = search.by_zipcode(zipcode)
    community = zipcode.post_office_city
    return community, "{},{}".format(zipcode.lat, zipcode.lng)


In [105]:
%%timeit
jobs["Community"] = ""
jobs["Coordinates"] = ""
for zipcode in jobs.Zip5.unique():
    community, coordinate = coordinates(zipcode)
    jobs.loc[jobs.Zip5 == zipcode, "Coordinates"] =  str(coordinate)
    jobs.loc[jobs.Zip5 == zipcode, "Community"] =  community

8.32 s ± 152 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
jobs.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
400,"New York, NY",10627
491,"Rochester, NY",1569
75,"Buffalo, NY",1256
73,"Brooklyn, NY",1030
70,"Bronx, NY",834
361,"Melville, NY",730
2,"Albany, NY",551
632,"White Plains, NY",487
564,"Syracuse, NY",459
249,"Hauppauge, NY",395


In [125]:
jobs.groupby(['Community', 'Zip5', 'Coordinates']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Zip5,Coordinates,Count
537,"New York, NY",10001,"40.75,-73.99",3302
551,"New York, NY",10017,"40.75,-73.973",1038
550,"New York, NY",10016,"40.74,-73.97",806
492,"Melville, NY",11747,"40.79,-73.4",730
570,"New York, NY",10036,"40.76,-73.99",638
552,"New York, NY",10018,"40.76,-73.99",537
556,"New York, NY",10022,"40.76,-73.97",486
541,"New York, NY",10005,"40.705,-74.005",416
369,"Hauppauge, NY",11788,"40.82,-73.21",395
589,"New York, NY",10167,"40.755,-73.975",390


### creating user coordinates

In [136]:
users = users.loc[users.State == 'NY']

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
28,682,1,Train,Mount Kisco,NY,US,10549,High School,Not Applicable,2005-01-01 00:00:00,4,7.0,Yes,No,0
50,1355,1,Train,Bronx,NY,US,10466,Associate's,"Heating, Air Conditioning and Refrigeration Te...",2011-12-01 00:00:00,3,10.0,NaN,No,0
51,1363,1,Train,Rochester,NY,US,14626,Bachelor's,Rhetoric & Communications,2011-12-01 00:00:00,3,4.0,Yes,No,0
62,1721,1,Train,Staten Island,NY,US,10310,High School,Not Applicable,NaN,4,19.0,Yes,No,0


In [138]:

def creating_coordinates(df, column):
    df["Community"] = ""
    df["Coordinates"] = ""
    for zipcode in df[column].unique():
        community, coordinate = coordinates(zipcode)
        df.loc[df[column] == zipcode, "Coordinates"] =  str(coordinate)
        df.loc[df[column] == zipcode, "Community"] =  community
    return df
users = creating_coordinates(users, 'ZipCode')

In [140]:
users.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
85,"Brooklyn, NY",2844
496,"New York, NY",2069
82,"Bronx, NY",1916
608,"Rochester, NY",1018
689,"Staten Island, NY",468
356,"Jamaica, NY",420
88,"Buffalo, NY",321
801,"Yonkers, NY",292
239,"Flushing, NY",190
30,"Astoria, NY",179


### Distance Matrix 

In [142]:
users.head(2)

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0,"Astoria, NY","40.78,-73.91"
28,682,1,Train,Mount Kisco,NY,US,10549,High School,Not Applicable,2005-01-01 00:00:00,4,7.0,Yes,No,0,"Mount Kisco, NY","41.2,-73.72"


In [143]:
jobs.head(2)

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate,Community,Coordinates
349,1285,1,SQL DBA,must be living in new york and be a u s citize...,education bachelor s degree in electrical engi...,New York,NY,US,0,2012-03-19 17:09:52.757,2012-04-18 23:59:59,None,"None,None"
350,1286,1,Customer Service Representative,superior staff resources is currently seeking ...,required skills qualifications nhigh school di...,Albany,NY,US,12203,2012-04-09 10:26:07.937,2012-05-08 23:59:59,"Albany, NY","42.68,-73.83"


In [145]:
apps.head(2)

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009


In [146]:
user_history.head(2)

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer


### Subset most applied jobs by title

In [321]:
top_50_by_history = user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False).head(top)
top_50_by_history.head(3)

,JobTitle,Count
156590,Customer Service Representative,19672
98604,Cashier,16368
22731,Administrative Assistant,16228


### Looking similar jobs

In [372]:
current_app_intercep = np.intersect1d(users.UserID.unique(), apps.UserID.unique())
historical_app_intercep = np.intersect1d(users.UserID.unique(), user_history.UserID.unique())
total_users = users.UserID.unique()
with_info =  set(current_app_intercep.tolist() + historical_app_intercep.tolist())

current_app = 1-len(current_app_intercep )/len(total_users )
historical_app = 1-len(historical_app_intercep)/ len(total_users )
cold_star_p = len(with_info)/len(total_users )

print(current_app_cold_star, historical_app_cold_star, cold_star_p)


0.17532782904322486 0.034677027683341466 0.9942690626517727


In [338]:
len(jobs.Title.unique())
len(top_50_by_history.JobTitle)
jobs.groupby(
    ['Title']).size().reset_index(
    name='Count').sort_values('Count', ascending=False).head(10000).Count.sum()
#10000 is almost the 100%

#Take the top 10 by top 1000


33493

In [ ]:
distance_jobs_df = pd.DataFrame(columns=jobs.JobID.unique(), index = users.index)

### Looks for most applied jobs id

In [322]:
top = 50
top_50_by_apps = apps.groupby(['JobID']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(top)
top_50_by_apps.head(3)

,JobID,Count
6875,17361,208
295156,900797,203
22755,67239,189


In [164]:
liked_jobs['JobTitle'].iloc[0]

'Customer Service Representative'

In [166]:
jobs_counter = jobs.groupby(['Title', 'Coordinates']).size().reset_index(name='Count').sort_values('Count', ascending=False)


### Distance matrix

In [284]:
c1 = jobs.Coordinates.iloc[1].split(',')
c2 = users.Coordinates.iloc[0].split(',')

distance_jobs_df = pd.DataFrame(columns=jobs.JobID.unique(), index = users.index)

,1285,1286,1306,1348,1423,1459,1540,1664,1709,1730,...,1114880,1115288,1115719,1115793,1115799,1115981,1116070,1116153,1116242,1116307
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
job = (jobs_counter[jobs_counter['Title']==liked_jobs['JobTitle'].iloc[top_j]]).iloc[title_i]
#recommender_by_liked.append(job)
liked_jobs



Title          Sales Associate
Coordinates       40.75,-73.88
Count                        4
Name: 30743, dtype: object

In [285]:
distance_jobs_df.to_csv()

KeyboardInterrupt: 

In [ ]:
### User x Jobs x Distance
for user in users:
    for job in jobs:
        c1 = jobs.Coordinates.iloc[1].split(',')
        c2 = users.Coordinates.iloc[0].split(',')

print (round(distance(c1, c2).miles, 2))
        update distance matrix
        
        

### Recomender top k liked

In [374]:
user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False).head(3)

,JobTitle,Count
156590,Customer Service Representative,19672
98604,Cashier,16368
22731,Administrative Assistant,16228


In [376]:
jobs.columns

Index(['JobID', 'WindowID', 'Title', 'Description', 'Requirements', 'City',
       'State', 'Country', 'Zip5', 'StartDate', 'EndDate', 'Community',
       'Coordinates'],
      dtype='object')

In [379]:
jobs.groupby(
    ['Title', 'Coordinates']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)

,Title,Coordinates,Count
24006,Own Your Own Franchise!,"None,None",296
17703,Inventory Taker,"None,None",185
22140,Mobile Tool Sales / Franchise Distributor,"None,None",142
31170,Sales Representative,"None,None",136
33408,Shift Supervisor,"None,None",124
27,(BSRO) - Automotive Lead Technician,"None,None",115
4716,Benefits Consultant,"None,None",89
9195,"Customer Account Representative, Now Accepting...","None,None",88
18010,Java/J2ee Developer,"40.75,-73.99",86
38255,j2ee Web Application Developer,"40.75,-73.99",86


In [398]:
jobs.columns

Index(['JobID', 'WindowID', 'Title', 'Description', 'Requirements', 'City',
       'State', 'Country', 'Zip5', 'StartDate', 'EndDate', 'Community',
       'Coordinates'],
      dtype='object')

In [435]:
unique = 0.3 #update name
top = 20

def ranking_by_popularity(top):
    popular_jobs = user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)
    ranking =  dict()
    top_i = 0
    while True:
        job_title = popular_jobs['JobTitle'].iloc[top_i]
        jobs_list = jobs.loc[jobs['Title'] == job_title, ['JobID']]['JobID'].unique().tolist()

        if len(jobs_list) > 1:
            ranking[job_title] = jobs_list

        if len(ranking) == top:
            break

        top_i +=1
    return ranking

ranking_popular = ranking_by_popularity(top)

Byr


In [439]:
def recommender_popular_jobs(unique, top):
    for title, jobs in ranking_popular.items():
        
        print(title, jobs)
        print()

In [440]:
recommender_popular_jobs(unique, top)

Customer Service Representative [1286, 45221, 81985, 114603, 166448, 167048, 179806, 251827, 269842, 281541, 297127, 326664, 411934, 413800, 439449, 439450, 463566, 465726, 466749, 467025, 563680, 565289, 570367, 645244, 711541, 717681, 757626, 766361, 766984, 1015707, 1020509, 1020694, 1038330, 189005, 418335, 419762, 457079, 466587, 570133, 663645, 1020507, 1020841, 1021191, 1094681, 139713, 167197, 285661, 339154, 570160, 719627, 721402, 842991, 862003, 869954, 874934, 901301, 971145, 1101316, 118506, 125463, 167663, 420718, 424755, 467397, 467595, 620972, 756929, 766515, 825101, 870268, 972723, 1021437, 1021439, 1049590, 127898, 167887, 419760, 429584, 466522, 467973, 587383, 635970, 767282, 767840, 911376, 925088, 1019998, 1021739, 73432, 127578, 129667, 166696, 167855, 321247, 465731, 468115, 724965, 774035, 917817, 1001278, 1081608, 68682, 127767, 166902, 167564, 167891, 277750, 508674, 514753, 589291, 657967, 767844, 806780, 819507, 1021544, 1077010]

Cashier [391531, 91129, 96

In [ ]:
def recommender_popular_jobs(unique, top):
     available_jobs = jobs.groupby(
    ['Title',  'Coordinates']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)
    
    top_j = 0
    title_i = 0
    ranking =  list()
    
    for recom in range(top):
        try:
            job = (available_jobs[available_jobs['Title']==popular_jobs['JobTitle'].iloc[top_j]]).iloc[title_i]
            ranking.append(job)
            title_i += 1
            if sum([recom, 1]) % int(unique*top) == 0:
                top_j += 1
                title_i = 0
        except:
            top_j += 1
            title_i = 0